\
            # 56. 机器学习基础：scikit-learn（sklearn）（scikit-learn Fundamentals）

            目标：掌握用 sklearn 完成一个典型机器学习项目的最小闭环：
- 数据集与特征/标签
- 训练/验证拆分、避免数据泄漏
- Pipeline（预处理+模型）
- 评估指标、交叉验证、调参
- 模型保存与上线前检查（概念）
本章依赖 `scikit-learn`（导入包名为 sklearn）；若未安装会提示安装命令并跳过相关演示。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- NumPy/Pandas 基础（建议）
- RESTful/缓存/工程化基础（了解更佳）


## 知识点地图

- 1. sklearn 心智模型：Estimator / Transformer / Pipeline
- 2. 安装与导入（可选依赖）
- 3. 数据集：用 Iris 演示分类问题（最小闭环）
- 4. 数据泄漏（核心坑）：为什么要用 Pipeline
- 5. Pipeline：StandardScaler + LogisticRegression
- 6. 评估：交叉验证（cross_val_score）
- 7. 调参：GridSearchCV（入门）
- 8. 模型保存：pickle/joblib（概念 + 可选代码）


## 自检清单（学完打勾）

- [ ] 理解监督学习的基本对象：X（特征）、y（标签）
- [ ] 会做 train/test split 并设置 random_state
- [ ] 会用 Pipeline 把预处理与模型绑在一起（避免泄漏）
- [ ] 会用常见指标评估分类/回归（accuracy/F1/MSE 等）
- [ ] 会做交叉验证与简单网格搜索（GridSearchCV）
- [ ] 知道模型持久化（保存/加载）与版本管理要点（概念）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：sklearn 心智模型：Estimator / Transformer / Pipeline

sklearn 的核心接口统一：
- Estimator：`fit(X, y)` 训练
- Predictor：`predict(X)` 预测（很多 estimator 同时是 predictor）
- Transformer：`fit_transform(X)` 做特征变换（StandardScaler、OneHotEncoder...）

Pipeline 把多个步骤串起来：
- 预处理（transform）
- 模型（predict）

价值：可复现、可交叉验证、避免数据泄漏。


## 知识点 2：安装与导入（可选依赖）

安装：
- `pip install scikit-learn`

注意：包名是 `scikit-learn`，导入名是 `sklearn`。


In [ ]:
try:
    import sklearn
except Exception as e:
    sklearn = None
    print('sklearn not available:', type(e).__name__, e)
    print('install: pip install scikit-learn')
else:
    print('sklearn version:', sklearn.__version__)


## 知识点 3：数据集：用 Iris 演示分类问题（最小闭环）

用内置 iris 数据集：
- X：四个花萼/花瓣特征
- y：三类鸢尾花

流程：
1) 拆分训练/测试
2) 训练模型
3) 评估指标


In [ ]:
try:
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, accuracy_score
except Exception as e:
    print('install scikit-learn to run this cell:', type(e).__name__, e)
else:
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

    clf = LogisticRegression(max_iter=200)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)

    print('accuracy:', round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))


## 知识点 4：数据泄漏（核心坑）：为什么要用 Pipeline

数据泄漏：把“测试集信息”不小心用进训练过程，导致评估虚高。

典型泄漏：
- 在全量数据上 fit scaler/encoder，然后再 split

正确做法：
- 把 scaler/encoder 放进 Pipeline
- 交叉验证/网格搜索时也对每个 fold 单独 fit


## 知识点 5：Pipeline：StandardScaler + LogisticRegression

标准化 + 线性模型是很常见的 baseline。

注意：
- `StandardScaler` 必须只在训练数据上 fit。
- Pipeline 会自动保证这一点。


In [ ]:
try:
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
except Exception as e:
    print('install scikit-learn to run this cell:', type(e).__name__, e)
else:
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=200)),
    ])
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    print('accuracy:', round(accuracy_score(y_test, pred), 4))


## 知识点 6：评估：交叉验证（cross_val_score）

一次 train/test split 的结果可能不稳定。

交叉验证（CV）：
- 把训练数据分成 k 份
- 轮流用 k-1 份训练、1 份验证
- 得到更稳健的评估分数


In [ ]:
try:
    from sklearn.datasets import load_iris
    from sklearn.model_selection import cross_val_score
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
except Exception as e:
    print('install scikit-learn to run this cell:', type(e).__name__, e)
else:
    X, y = load_iris(return_X_y=True)
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=300)),
    ])
    scores = cross_val_score(pipe, X, y, cv=5)
    print('cv scores:', scores)
    print('mean:', scores.mean(), 'std:', scores.std())


## 知识点 7：调参：GridSearchCV（入门）

调参要点：
- 参数搜索在训练集内部进行（CV），不要偷看测试集。
- 搜索空间不要太大：先粗后细。

示例：对 C（正则强度）做网格搜索。


In [ ]:
try:
    from sklearn.datasets import load_iris
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
except Exception as e:
    print('install scikit-learn to run this cell:', type(e).__name__, e)
else:
    X, y = load_iris(return_X_y=True)
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=500)),
    ])

    gs = GridSearchCV(
        pipe,
        param_grid={'clf__C': [0.1, 1.0, 10.0]},
        cv=5,
        n_jobs=None,
    )
    gs.fit(X, y)
    print('best params:', gs.best_params_)
    print('best score:', gs.best_score_)


## 知识点 8：模型保存：pickle/joblib（概念 + 可选代码）

模型上线需要：
- 保存训练好的 pipeline（包含预处理）
- 记录版本：数据版本/代码版本/参数版本
- 上线前校验：输入 schema、输出范围、回归测试

保存方式：
- `joblib.dump`（sklearn 官方常用）
- `pickle`（通用，但注意安全：不要加载不可信 pickle）


In [ ]:
from pathlib import Path

try:
    import joblib
    from sklearn.datasets import load_iris
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
except Exception as e:
    print('optional deps missing:', type(e).__name__, e)
    print('install: pip install scikit-learn joblib')
else:
    X, y = load_iris(return_X_y=True)
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=300)),
    ])
    pipe.fit(X, y)

    ART = Path('_nb_artifacts')
    ART.mkdir(exist_ok=True)
    path = ART / 'iris_pipe.joblib'
    joblib.dump(pipe, path)
    print('saved to', path)

    loaded = joblib.load(path)
    print('predict first 3:', loaded.predict(X[:3]))


## 常见坑

- 数据泄漏：预处理在全量数据上 fit
- 只看训练集分数：没做验证/测试，过拟合看不出来
- 调参偷看测试集：导致指标虚高
- merge 数据后行数不对：标签对齐错误（常见于表连接）
- 模型持久化加载不安全：不要加载不可信 pickle/joblib


## 综合小案例：构建一个“可上线”的分类 Pipeline（含输入校验思路）

目标：
- 选一个数据集（iris 或你自己的 CSV）
- 构建 pipeline：预处理 + 模型
- 用 CV 评估并调参
- 保存模型到 `_nb_artifacts`
- 写一份“上线检查清单”：
  - 输入字段/类型/范围校验
  - 版本号（模型/数据/特征）
  - 监控指标（延迟、错误率、分布漂移）


In [ ]:
# 建议：把检查清单写成 Markdown 文档并放进项目仓库。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- 为什么推荐使用 Pipeline？它如何帮助避免数据泄漏？
- 交叉验证解决了什么问题？
- GridSearchCV 应该在什么时候用？为什么不应该在测试集上调参？
- 为什么上线时要保存“预处理 + 模型”的整体？


## 练习题（建议写代码）

- 把分类任务换成回归（例如 Boston 已弃用，可用自造数据或 diabetes 数据集），并使用 MSE/MAE 评估。
- 为 Pipeline 加入特征选择（SelectKBest）并比较效果（了解）。
- 写一个 RESTful 预测接口（FastAPI/Flask）加载 joblib 模型并提供 /predict（与框架章节呼应）。
